Clustering is one of the ways for unsupervised learning. It is useful when we want to see the internal groupings of an unlabeled data set. The **goal** of this notebook is to introduce the **basic clustering approaches** and their **implementations in Python**. 


# K-means clustering


## How does it work

* Guess some cluster centers
* Repeat until converged
 - E-Step: assign points to the nearest cluster center
 - M-Step: set the cluster centers to the mean


<figure>
 <img src="https://miro.medium.com/max/960/1*KrcZK0xYgTa4qFrVr0fO2w.gif" alt="K-means process" />
 <figcaption>
  K-means clustering in progress <a href="https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68">The 5 Clustering Algorithms Data Scientists Need to Know</a>
 </figcaption>
</figure>




## Assumptions (it works when...?)
Clusters should be spherical, well separated, have similar volumes and similar number of points.


We will be working on a toy dataset of delivery fleet driver (from [Introduction to K-means Clustering](https://https://www.datascience.com/blog/k-means-clustering)). Let's load the data and take a look first (run the following code chunk):


In [0]:
# Import delivery fleet driver data
## Import pandas. Used here for file loading. 
## This package is widely used for data manipulation and analysis.
import pandas as pd 

url = 'https://raw.githubusercontent.com/datascienceinc/learn-data-science/master/Introduction-to-K-means-Clustering/Data/data_1024.csv'
# The URL above can be replaced by any source of a raw csv-format dataset

data = pd.read_csv(url, sep = "\t") # Read from the URL source as file.

print(data.shape) # Show the dimsension of data 
data.head() # See the top 5 rows

So our data set contains Driver_ID, Distance, and Speeding information for 4000 drivers -> a 4000 rows by 2 columns matrix. We can describe the drivers by their distance and speeding features:

In [0]:
# Import modules for plotting and plotting styling
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

In [0]:
# Plot Distance as x-axis and Speeding as y-axis for drivers
plt.scatter(data["Distance_Feature"], data["Speeding_Feature"])
plt.xlabel("Distance")
plt.ylabel("Speeding")
plt.show()

With naked eye, we can see roughly two to four groups of drivers, separated by their Distance and Speeding. Let's see how good can K-means do.

In [0]:
# Import modules required for Kmeans calculation
import numpy as np # Used for managing arrays
from sklearn.cluster import KMeans 

## Data reformatting 
We used pandas to read our csv file in, and by-default, it is stored as a pandas dataframe

In [0]:
type(data)

However, this format is not what KMeans can take. We need to reformat to meet the input requirement.

In [0]:
# Formatting data
## scikit learn kmeans requires the input as ndarray.
## transform the pandas dataframe into arrays
## extract values from the columns 1 - Distance, 2 - Speeding 
X = data.iloc[:,1:3].values  
print(type(X))
X

## Run K-means: interpretations and optimizations
Now, we can run the data set as an ndarray. Notice, k-means requires that we provide the number of clusters. From what we see, we can try two first.

In [0]:
# Use reformatted data X for K-means calculation
kmeans = KMeans(n_clusters=2).fit(X)
# Clustering assignments can be retrieved from .label_ .
# apply this assignment to the Speeding-by-Distance plot
plt.scatter(data["Distance_Feature"], data["Speeding_Feature"], 
            c=kmeans.labels_, cmap='viridis')
# Plot the cluster centers on top
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5) 
plt.xlabel("Distance")
plt.ylabel("Speeding")
plt.show()

Looks like the data grouped well with `n_clusters=2`. But we do see separations within each clusters. What would the optimal number for clustering this dataset and how would we know?

Run the code chunk below, and see if you can find the number that works the best for this dataset (play around a couple of values for number of clusters).


In [0]:
# Test different numbers of k for the k-means result
# Import interactive modules 
from ipywidgets import interact
import ipywidgets as widgets
# Define function to run interactively. Here we allow the number of clusters 
# to be an interactive element in the KMeans calls.
def KMeans_test_n_cl(num_clusters):
    kmeans = KMeans(n_clusters=num_clusters).fit(X)
    # Clustering assignments can be retrieved from .label_ .
    # apply this assignment to the Speeding-by-Distance plot
    plt.scatter(data["Distance_Feature"], data["Speeding_Feature"], 
                c=kmeans.labels_, cmap='viridis')
    # Plot the cluster centers on top
    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5) 
    plt.xlabel("Distance")
    plt.ylabel("Speeding")
    plt.show()
    return
# Run interact
interact(KMeans_test_n_cl, 
         num_clusters=widgets.IntSlider(min=2, max=10, step=1, value=2))

Testing manually could be annoying when dealing with more complex datasets. One automatic optimization approach would be the **silhouette analysis**. Ranging from [-1,1], the silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). Higher value indicates that the object is well matched to its own cluster. If most objects have a high value, then the clustering configuration is appropriate ([wiki/Silhouette_(clustering)](https://https://en.wikipedia.org/wiki/Silhouette_(clustering))).

In [0]:
# Import modules for silhouette calculations
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

# We'll just test within the range of [2,6]. 
# Any range you are interested in testing could replace this
for n_clusters in range(2,6):
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    kmeans = clusterer.fit(X)
    cluster_labels = kmeans.labels_
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters 
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)


    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
plt.show()

A good number of clusters should meet the following:
1. The mean value should be as close to 1 as possible
2. The plot of each cluster should be above the mean value as much as possible. Any plot region below the mean value is not desirable.
3. The width of the plot should be as uniform as possible.

(from [USING SILHOUETTE ANALYSIS FOR SELECTING THE NUMBER OF CLUSTER FOR K-MEANS CLUSTERING](http://kapilddatascience.wordpress.com/2015/11/10/using-silhouette-analysis-for-selecting-the-number-of-cluster-for-k-means-clustering/))

Although four-cluster assignment does look better, two-cluster assignment got the highest score. This could be due the fact that the data set itself does not fit the assumptions that well. 

In [0]:
# If only interested in the average silhouette score: 
for n_clusters in range(2,6):
  # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    kmeans = clusterer.fit(X)
    cluster_labels = kmeans.labels_
    
    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters 
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)


## Ending notes for K-means
People love k-means for its speedity and simplicity (when the sample number is small). But there are a couple of things not so great for k-means:
1. it can get stuck in local minimals due to bias in initiation
2. users need to make the decision of how many clusters should in the data
3. it assumes clusters to be spherical, well-separated, similar in volume and the number of samples they contain

To avoid (1), it is often recommended to run the calculation multiple times (scikit learn is doing it by default, so no worries :tada:). As for (2), we can run silhouette analysis for estimation. 

But if you are dealing with (3), there seem to be no workarounds to make k-means to give a better result. One quick alternative to try is the Gaussian mixture models (GMM).

# Gaussian mixture model
Every sample in k-means belongs and only belongs to one cluster, which means there is no room for uncertainty. That's why k-means is also called a "hard" clustering method. What if we take a softer approach?

## Generalizing K-means
A Gaussian mixture model (GMM) is a **probabilistic model** that assumes all the data points are generated from a mixture of a finite number of Gaussian distributions with unknown parameters. In a way, GMM is generaling the k-means model.

<figure>
 <img src="https://miro.medium.com/max/1400/1*lTv7e4Cdlp738X_WFZyZHA.png" alt="Gaussian mixture model" width="500"/>
 <figcaption>
  Graphical illustration of Gaussian mixture model from <a href="https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95">Gaussian Mixture Models Explained</a>
 </figcaption>
</figure>

<figure>
 <img src="https://miro.medium.com/max/1750/1*I0WTzTOyyDVwfPyMSZPzWQ.gif" alt="GMM in progress" width="500"/>
 <figcaption>
  GMM clustering in progress from <a href="https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95">Gaussian Mixture Models Explained</a>
 </figcaption>
</figure>


Let's see if this can work better than k-means for our data.


## Run GMM: interpretations and optimizations

In [0]:
# Import GMM module
from sklearn.mixture import GaussianMixture
# Specify number of clusters/components, and fit the model to data
# the same reformatted data X is used
gmm = GaussianMixture(n_components=4).fit(X) 
# obtain labels for X
labels = gmm.predict(X)
# color samples with the cluster labels
plt.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis')

Since GMM uses probabilistic models, we can also visualize the clustering assignment "strength" of each sample. Here we'll show higher probability as large size.

In [0]:
# obtain the probability matrix from GMM
probs = gmm.predict_proba(X)
# convert probability into a size feature
size = 50 * probs.max(1) ** 2  # square emphasizes differences
# coloring dots by clustering assignment and tuning their size based on the
# probability of their belonging to the cluster
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=size) 

Note that GMM also requires a **prior estimation of number of clusters** (`n_components`), which leads to the same question of how would we know if this is correct? For GMM, we can use the criteria of Akaike information criterion (**AIC**) or the Bayesian information criterion (**BIC**). These are both included as attributes in the scikit learn GMM calculations.

In [0]:
# Calculate AIC and BIC for number of clusters from 1 to 10
n_components = np.arange(1, 11)
# create a list of models each use a different cluster number
models = [GaussianMixture(n, covariance_type='full', random_state=0).fit(X)
          for n in n_components]
# plot AIC and BIC trends versus number of clusters
plt.plot(n_components, [m.bic(X) for m in models], label='AIC')
plt.plot(n_components, [m.aic(X) for m in models], label='BIC')
plt.legend(loc='best')
plt.xlabel('n_components');

The optimal number of clusters is the one that **minimizes AIC or BIC value**. This time, four-component clustering does give us the best performance. Running an AIC or BIC estimation will allow you to pick the right number of clusters/components to use in the GMM calculation. This estimation itself can also be handled automatically if you use  **Bayesian Gaussian mixture model**, which implemented variational inference to infer the effective number of components from the data.

In [0]:
# Import Bayesian Gaussian mixture model
from sklearn.mixture import BayesianGaussianMixture
# the same reformatted data X is used
bgmm = BayesianGaussianMixture(n_components=5, n_init=10).fit(X) 
# obtain labels for X
labels = bgmm.predict(X)
# show number of clusters in the assignment
print("Number of clusters: %2d" %(len(np.unique(labels))))
# color samples with the cluster labels
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')

We gave Bayesian GMM `n_components=5`. Instead of using all to fit the data, as what GMM would do, the Bayesian model effectively uses as many as are needed for a good fit (when the end result converges). As a result, we got the four-cluster assignment as in the optimized GMM.

## Density estimations

So far, what we've encountered have all required us to have prior knowledge about number of clusters. At the same time, they do not perform well when the shape of the clusters largely deviate from an ellipse (e.g., crescent moon). Let's see how GMM perform on this type of data (example from [In Depth: Gaussian Mixture Model](http://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html)):


In [0]:
# Let's make some moons
from sklearn.datasets import make_moons
Xmoon, ymoon = make_moons(200, noise=.05, random_state=0)
plt.scatter(Xmoon[:, 0], Xmoon[:, 1]);

In [0]:
# Define functions to show us the GMM result more intuitively
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))
        
def plot_gmm(gmm, X, label=True, ax=None):
    ax = ax or plt.gca()
    labels = gmm.fit(X).predict(X)
    if label:
        ax.scatter(X[:, 0], X[:, 1], c=labels, s=40, cmap='viridis', zorder=2)
    else:
        ax.scatter(X[:, 0], X[:, 1], s=40, zorder=2)
    ax.axis('equal')
    
    w_factor = 0.2 / gmm.weights_.max()
    for pos, covar, w in zip(gmm.means_, gmm.covariances_, gmm.weights_):
        draw_ellipse(pos, covar, alpha=w * w_factor)

In [0]:
# Fit with 2-component GMM 
gmm2 = GaussianMixture(n_components=2, covariance_type='full', random_state=0)
plot_gmm(gmm2, Xmoon)

Although we have a clear estimate of total number of clusters should be there in the data, the returned result from GMM is not very useful. But if we instead use many more components and ignore the cluster labels, we find a fit that is much closer to the input data: 

In [0]:
# Fit with 16-component GMM
gmm16 = GaussianMixture(n_components=16, covariance_type='full', random_state=0)
plot_gmm(gmm16, Xmoon, label=False)

Here the mixture of 16 Gaussians serves not to find separated clusters of data, but rather to model the overall distribution of the input data. The GMM algorithm accomplishes this by representing the density as a weighted sum of Gaussian distributions. What if we push to the extreme and use the density feature of data to facilitate clustering?

# DBSCAN
DBSCAN stands for Density-Based Spatial Clustering of Applications with Noise, which is one of the ways taking advantage of density in data for clustering. It shares a lot in its logic with [Kernel density estimation (KDE)](https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html), but uses different appraoch in the density estimation, thus less expensive to compute. We will only cover DBSCAN in this notebook, but feel free to also check out the link embedded above for detailed explanations for the general idea of KDE. 

<figure>
 <img src="https://miro.medium.com/max/1688/1*tc8UF-h0nQqUfLC8-0uInQ.gif" alt="DBSCAN on smiley face" width="500"/>
 <figcaption>
  DBSCAN smiley face clustering in progress (source from <a href="https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68">The 5 Clustering Algorithms Data Scientists Need to Know</a>). For clustering in real time, check out <a href="https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/?source=post_page---------------------------">Visualizing DBSCAN Clustering</a>. 
 </figcaption>
</figure>

Great things about **DBSCAN** include:
1. no requirement for a pre-set number of clusters
2. can discover clusters of arbitrary shapes
3. efficient for large data set (over a couple thousand)

## How does it work?

1. Arbitrary initiation, then retrieve all neighborhood information within distance ϵ.
2. If this point contains MinPts within ϵ neighborhood, cluster formation starts. Otherwise the point is labeled as noise. This point can be later found within the ϵ neighborhood of a different point and, thus can be made a part of the cluster.
3. points within the neighborhood of the first point become part of the cluster. So all the points found within ϵ neighborhood are added, along with their own ϵ neighborhood, if they themselves also contains MinPts within their ϵ neighborhood.
4. The above process continues until the density-connected cluster is completely found.
5. The process restarts with a new point which can be a part of a new cluster or labeled as noise.



In [0]:
# Import DBSCAN module
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

## Run DBSCAN: interpretations and optimizations

In [0]:
# code credit from 
# https://medium.com/@elutins/dbscan-what-is-it-when-to-use-it-how-to-use-it-8bd506293818
# Scale the data first
scaledX = StandardScaler().fit_transform(X)

DBSCAN actually still require some prior knowledge of the data to set the two parameters: distance ϵ and threshold for number of points (minPts). Let's see how these two could affect the result:

In [0]:
def DBSCAN_test_eps_minpts(eps=0.1, minPts = 1):
  # Setting up DBSCAN
  dbscan = DBSCAN(eps = eps, min_samples=minPts)
  # Fitting model
  model = dbscan.fit(scaledX)
  labels = model.labels_

  from sklearn import metrics
  # identifying the core samples
  core_samples = np.zeros_like(labels, dtype=bool)

  # core_samples[dbscan.core_sample_indices_] = True
  # print(core_samples)

  # declare the number of clusters
  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  print("Number of clusters found: %d" % n_clusters_)

  # computing the Silouette Score
  print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(scaledX, labels))
  
  # Plot 
  color_palette = sns.color_palette('Paired', 10)
  cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in model.labels_]
  plt.scatter(scaledX[:,0],scaledX[:,1], c=cluster_colors, alpha=0.3)
  plt.show()
  return

interact(DBSCAN_test_eps_minpts,
        eps = (0.1,1.0), 
        minPts = widgets.IntSlider(min=1,max=10, step=1, value=10))


We again use the Silhouette score to measure the clustering performance. Gray dots in this plot are noises defined by DBSCAN. If we follow the recommended condition `minPts = 4`, we can see that there seem to be no right value for epsilon for a nice clustering result. As epsilon gets smaller, we start to see DBSCAN calling numerous random clusters from the noisy upper right corner.  

This unsatisfactory performance goes back to the question of whether our data meets the assumptions made by the algorithm. DBSCAN is resistant to noise and can handle clusters that come in different shape and sizes. However, when clusters have varying densities, which is what our data looks, it does not work well.




>  Side note: Finding the best values for minPts and epsilon?
>  - minPts
1. For two-dimensional data: use default value of minPts=4 (Ester et al., 1996)
2. For more than 2 dimensions: minPts=2*dim (Sander et al., 1998)
>
> - Estimate epsilon based on minPts
 1. Plot the k-distances with k=minPts (Ester et al., 1996)
 2. Find the 'elbow' in the graph--> The k-distance value is the Epsilon value.

Is there a way that allows us to use density estimation for clustering without being affected by the density variation across clusters? Methods including OPTICS and HDBSCAN are all worth considering. I provide ways to implement these methods below for you to try. 

Spoiler: although these method claims to do better than DBSCAN in when clusters vary in densities, they might not give better results if the parameters are not optimized.

## OPTICS

OPTICS stands for Ordering Points To Identify the Clustering Structure, a close relative of DBSCAN. It finds high density sample cores, and expands from there to form clusters. Cluster extraction step could still be done with the DBSCAN approach or more automatically by the "xi" approach [sklearn OPTICS](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html).
OPTICS does a better job in keeping cluster hierarchy for a variable neighborhood radius, thus handles clusters with varying densities better.

In [0]:
from sklearn.cluster import OPTICS
clust = OPTICS(min_samples=4, cluster_method='dbscan', eps = 0.2) # 

# Run the fit
labels = clust.fit_predict(scaledX)
color_palette = sns.color_palette('Paired', 10)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in labels]
plt.scatter(scaledX[:,0],scaledX[:,1], c=cluster_colors, alpha=0.3)
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("Number of clusters: %d" % (n_clusters))

## HDBSCAN


HDBSCAN is an extended version of DBSCAN, converting it into  a hierarchical clustering algorithm, and then using a technique to extract a flat clustering based in the stability of clusters. See [Using HDBSCAN](https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html).

In [0]:
!pip install hdbscan
import hdbscan

In [0]:
clust = hdbscan.HDBSCAN(min_cluster_size=100, min_samples=4)

# Run the fit
clust.fit(scaledX)
labels = clust.labels_

# Ignore noises in counting clusters
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("Number of clusters: %d" % (n_clusters))

# Set colors for plotting clusters
color_palette = sns.color_palette('Paired', 10)
cluster_colors = [color_palette[x] if x >= 0
                  else (0.5, 0.5, 0.5)
                  for x in labels]
plt.scatter(scaledX[:,0],scaledX[:,1], c=cluster_colors, alpha=0.25)

# Take-home messages

1. Know your data. Different density, shape, noise-level of the data could all affect the performance of clustering approaches.
2. Tune parameters for the better result. Default setting often times are not the setting to use.
3. Open to alternative clustering approaches.

Hungry for more? Check out [here](https://scikit-learn.org/stable/modules/clustering.html) for a nice summary of clustering approaches by scikit learn. ***Happy clustering!***



References:

1. https://courses.cs.washington.edu/courses/cse446/15sp/slides/cdr.pdf
2. [The 5 Clustering Algorithms Data Scientists Need to Know](https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68)
3. [Introduction to K-means Clustering](https://https://www.datascience.com/blog/k-means-clustering)
4. [wiki/Silhouette_(clustering)](https://https://en.wikipedia.org/wiki/Silhouette_(clustering))
5.  [USING SILHOUETTE ANALYSIS FOR SELECTING THE NUMBER OF CLUSTER FOR K-MEANS CLUSTERING](http://kapilddatascience.wordpress.com/2015/11/10/using-silhouette-analysis-for-selecting-the-number-of-cluster-for-k-means-clustering/)
6. [Gaussian Mixture Models Explained](https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95)
7. [In Depth: Gaussian Mixture Model](http://jakevdp.github.io/PythonDataScienceHandbook/05.12-gaussian-mixtures.html)
8. [Kernel density estimation (KDE)](https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html)
9. [Visualizing DBSCAN Clustering](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/?source=post_page---------------------------)

